In [1]:
import os
import pickle

import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
# Retrieve data 
data = pd.read_csv('prepped-data/train_full.csv')

In [3]:
data

,coupon_id,customer_id,redemption_status,age_range,marital_status,family_size,no_of_children,income_bracket,gender,mean_discount_used_by_cust,unique_items_bought_by_cust,mean_selling_price_paid_by_cust,mean_quantity_bought_by_cust,total_discount_used_by_cust,total_coupons_used_by_cust,total_price_paid_by_cust,total_quantity_bought_by_cust,mean_coupon_discount,mean_item_price,category
0,27,1053,0,3,1,1,0,5,1,-0.29,208,164.25,1.08,-89.05,1,50918.77,335,-1.13,118.41,Women
1,27,1053,0,3,1,1,0,5,1,-0.29,208,164.25,1.08,-89.05,1,50918.77,335,-1.13,118.41,Men
2,27,1053,0,3,1,1,0,5,1,-0.29,208,164.25,1.08,-89.05,1,50918.77,335,-1.13,118.41,Sport
3,27,1053,0,3,1,1,0,5,1,-0.29,208,164.25,1.08,-89.05,1,50918.77,335,-1.13,118.41,Girls
4,27,1053,0,3,1,1,0,5,1,-0.29,208,164.25,1.08,-89.05,1,50918.77,335,-1.13,118.41,Boys
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289612,681,623,0,2,1,1,0,2,0,0.00,239,106.05,1.13,0.00,0,75509.00,804,-1.96,169.80,Men
289613,681,623,0,2,1,1,0,2,0,0.00,239,106.05,1.13,0.00,0,75509.00,804,-1.96,169.80,Girls
289614,681,623,0,2,1,1,0,2,0,0.00,239,106.05,1.13,0.00,0,75509.00,804,-1.96,169.80,Sport
289615,681,623,0,2,1,1,0,2,0,0.00,239,106.05,1.13,0.00,0,75509.00,804,-1.96,169.80,Women


In [4]:
data.drop('redemption_status', axis=1, inplace=True)

In [5]:
data.drop(['mean_item_price', 'mean_coupon_discount'], axis=1, inplace=True)

In [6]:
ids = data[['coupon_id', 'customer_id']]

In [7]:
data.drop(['coupon_id', 'customer_id'], axis=1, inplace=True)

In [8]:
model_path = 'pickled_models/scikit_classifier'

In [9]:
with open(model_path, 'rb') as f:
    model = pickle.load(f)

In [10]:
encoder = LabelEncoder()
data['age_range'] = encoder.fit_transform(data['age_range'])
print('Age range encoding:')
for i, c in enumerate(encoder.classes_):
    print(f'{i}: {c}')

data['marital_status'] = encoder.fit_transform(data['marital_status'])
print('Marital status encoding:')
for i, c in enumerate(encoder.classes_):
    print(f'{i}: {c}')

data['gender'] = encoder.fit_transform(data['gender'])
print('Gender encoding:')
for i, c in enumerate(encoder.classes_):
    print(f'{i}: {c}')

data = pd.get_dummies(data, columns=['category'])

Age range encoding:
0: 0
1: 1
2: 2
3: 3
4: 4
5: 5
Marital status encoding:
0: 0
1: 1
Gender encoding:
0: 0
1: 1


In [11]:
data.columns

Index(['age_range', 'marital_status', 'family_size', 'no_of_children',
       'income_bracket', 'gender', 'mean_discount_used_by_cust',
       'unique_items_bought_by_cust', 'mean_selling_price_paid_by_cust',
       'mean_quantity_bought_by_cust', 'total_discount_used_by_cust',
       'total_coupons_used_by_cust', 'total_price_paid_by_cust',
       'total_quantity_bought_by_cust', 'category_Boys', 'category_Girls',
       'category_Men', 'category_Sport', 'category_Women'],
      dtype='object')

In [12]:
preds = model.predict(data)

In [13]:
data['prediction'] = preds

In [14]:
data['customer_id'] = ids['customer_id']
data['coupon_id'] = ids['coupon_id']

### Decode categories

In [15]:
encoded_labels = ['category_Boys', 'category_Girls', 'category_Men', 'category_Sport', 'category_Women']
category = data[encoded_labels].idxmax(axis=1).apply(lambda x: x.replace('category_', ''))
data['category'] = category
data.drop(encoded_labels, axis=1, inplace=True)
data.sample(5)

,age_range,marital_status,family_size,no_of_children,income_bracket,gender,mean_discount_used_by_cust,unique_items_bought_by_cust,mean_selling_price_paid_by_cust,mean_quantity_bought_by_cust,total_discount_used_by_cust,total_coupons_used_by_cust,total_price_paid_by_cust,total_quantity_bought_by_cust,prediction,customer_id,coupon_id,category
158603,3,1,1,0,10,0,0.00,315,91.65,1.08,0.00,0,42708.98,501,0,61,773,Men
171765,3,0,2,0,4,0,-0.07,130,113.74,1.00,-14.25,1,23998.79,211,0,489,734,Women
141977,1,0,3,1,6,1,-0.09,753,99.68,1.00,-94.39,3,102366.34,1028,0,324,846,Girls
110449,3,1,2,1,5,0,-0.02,697,110.31,1.02,-17.81,1,111413.62,1030,0,38,248,Sport
50595,2,0,3,1,12,1,-0.01,848,131.17,1.00,-12.47,1,223909.04,1707,0,253,810,Girls


In [16]:
data = data[['customer_id', 'coupon_id', 'prediction', 'category']]

In [17]:
cust_cat = data.loc[data['prediction'] == 1][['customer_id', 'category']].value_counts().rename('hit_count').reset_index().sort_values(by='customer_id')

In [18]:
cust_cat

,customer_id,category,hit_count
51,1,Men,78
229,12,Boys,38
270,40,Boys,22
84,45,Women,66
157,45,Boys,52
...,...,...,...
137,1573,Girls,55
124,1573,Boys,58
253,1578,Men,30
262,1578,Sport,27


In [19]:
cust_cat_stats = pd.pivot_table(cust_cat, index='customer_id', values=['category', 'hit_count'],
                                aggfunc={
                                    'category': lambda x: len(set(x)),
                                    'hit_count': sum
                                })
cust_cat_stats = cust_cat_stats.reset_index().sort_values(by=['category', 'hit_count'], ascending=False)

In [20]:
good_customers = cust_cat_stats.loc[cust_cat_stats['category'] == 3].customer_id

In [21]:
good_customers_preds = pd.merge(good_customers, data, on='customer_id', how='left')

In [22]:
assert len(good_customers) == len(good_customers_preds.customer_id.unique())

In [23]:
cust_hits = good_customers_preds.loc[good_customers_preds['prediction'] == 1]\
    .customer_id.value_counts().reset_index().rename(columns={'index': 'customer_id', 'customer_id': 'total_hits'})

In [24]:
good_customers_preds = pd.merge(good_customers_preds, cust_hits, on='customer_id', how='left')

In [25]:
good_customers_preds.category.value_counts()

Women    1299
Men      1229
Sport    1207
Boys      976
Girls     958
Name: category, dtype: int64

In [26]:
len(good_customers_preds.loc[good_customers_preds['total_hits'] > 100].customer_id.unique())

15

In [27]:
good_customers_preds = good_customers_preds.loc[good_customers_preds['total_hits'] > 100]

In [28]:
good_customers_preds_hits_only = good_customers_preds.loc[good_customers_preds['prediction'] == 1]
good_customers_preds_hits_only

,customer_id,coupon_id,prediction,category,total_hits
0,464,606,1,Sport,398
1,464,606,1,Women,398
2,464,606,1,Men,398
4,464,310,1,Sport,398
5,464,310,1,Men,398
...,...,...,...,...,...
5162,640,617,1,Men,120
5163,640,617,1,Women,120
5164,640,610,1,Women,120
5166,640,610,1,Men,120


## Save data to files

In [29]:
dirname = 'demo-data'
if not os.path.exists(dirname):
    os.mkdir(dirname)

In [30]:
good_customers_preds_hits_only[['customer_id', 'category']].drop_duplicates()\
    .to_csv(os.path.join(dirname, 'customers_categories.csv'), index=False)
cust_coup_cat = good_customers_preds_hits_only[['customer_id', 'coupon_id', 'category']]
cust_coup_cat.to_csv(os.path.join(dirname, 'customers_coupons_categories.csv'), index=False)

### Save with full details

In [31]:
# Retrieve original data
data = pd.read_csv('prepped-data/train_full.csv')
data.drop('redemption_status', axis=1, inplace=True)

In [32]:
details = pd.merge(cust_coup_cat, data, on=['customer_id', 'coupon_id', 'category'], how='left')
details.to_csv(os.path.join(dirname, 'cust_coup_cat_details.csv'))